# OOD Detection for Customer Intent Classification

## Resources

Datasets 
- [banking77](https://huggingface.co/datasets/banking77)
- [ATIS Airline Travel Information System](https://www.kaggle.com/datasets/hassanamin/atis-airlinetravelinformationsystem) 
- [Bitext - Customer Service Tagged Training Dataset for Intent Detection](https://github.com/bitext/customer-support-intent-detection-training-dataset)

Model
- [philschmid/BERT-Banking77](https://huggingface.co/philschmid/BERT-Banking77) 

BERT resources
- [BERT Neural Network - EXPLAINED!](https://www.youtube.com/watch?v=xI0HHN5XKDo) 
- [Transformer Neural Networks - EXPLAINED! (Attention is all you need)](https://www.youtube.com/watch?v=TQQlZhbC5ps)

OOD detection
- [`Todd`](https://github.com/icannos/Todd)
- [OOD Classification `ToddBenchmark`](https://github.com/icannos/ToddBenchmark/tree/master/examples/classifications) 

## Setup

### Import

In [1]:
import torch
from transformers import AutoConfig

In [2]:
from tqdm import tqdm
from typing import List

In [3]:
from Todd import ScorerType, MahalanobisScorer

In [4]:
from toddbenchmark.classification_datasets import (
    prep_model, 
    load_b77, 
)
from toddbenchmark.classification_datasets_configs import (
    DATASETS_CONFIGS,
    load_requested_dataset,
)

from toddbenchmark.utils_classification import (
    prepare_detectors,
    evaluate_dataloader,
)

from toddbenchmark.utils import mk_file_name

### Load model and tokenizer

In [5]:
MODELS = {"BERT": "philschmid/BERT-Banking77", "DistilBERT": "philschmid/DistilBERT-Banking77"} 

In [6]:
config = AutoConfig.from_pretrained(MODELS["BERT"]) 
config

BertConfig {
  "_name_or_path": "philschmid/BERT-Banking77",
  "_num_labels": 77,
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 1.000E-01,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 1.000E-01,
  "hidden_size": 768,
  "id2label": {
    "0": "Refund_not_showing_up",
    "1": "activate_my_card",
    "2": "age_limit",
    "3": "apple_pay_or_google_pay",
    "4": "atm_support",
    "5": "automatic_top_up",
    "6": "balance_not_updated_after_bank_transfer",
    "7": "balance_not_updated_after_cheque_or_cash_deposit",
    "8": "beneficiary_not_allowed",
    "9": "cancel_transfer",
    "10": "card_about_to_expire",
    "11": "card_acceptance",
    "12": "card_arrival",
    "13": "card_delivery_estimate",
    "14": "card_linking",
    "15": "card_not_working",
    "16": "card_payment_fee_charged",
    "17": "card_payment_not_recognised",
    "18": "card_payment_wrong_exchang

In [7]:
model, tokenizer = prep_model(MODELS["BERT"], config={"label": config._num_labels})

In [8]:
model 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

### Load reference dataset

In [9]:
train_loader, _, test_loader = load_requested_dataset(
    config_name="b77", 
    tokenizer=tokenizer, 
    train_size=100, 
    validation_size=0, 
    test_size=100
)

Found cached dataset banking77 (C:/Users/pemma/.cache/huggingface/datasets/banking77/default/1.1.0/ff44c4421d7e70aa810b0fa79d36908a38b87aff8125d002cd44f7fcd31f493c)
100%|██████████| 2/2 [00:00<00:00, 22.11it/s]
Loading cached processed dataset at C:\Users\pemma\.cache\huggingface\datasets\banking77\default\1.1.0\ff44c4421d7e70aa810b0fa79d36908a38b87aff8125d002cd44f7fcd31f493c\cache-789d791b63800d76.arrow
Loading cached processed dataset at C:\Users\pemma\.cache\huggingface\datasets\banking77\default\1.1.0\ff44c4421d7e70aa810b0fa79d36908a38b87aff8125d002cd44f7fcd31f493c\cache-f2f24408eb806cc9.arrow


In [10]:
for batch in train_loader: 
    break 

In [11]:
batch

{'text': ['I am still waiting on my card?',
  "What can I do if my card still hasn't arrived after 2 weeks?",
  'I have been waiting over a week. Is the card still coming?',
  'Can I track my card while it is in the process of delivery?',
  'How do I know if I will get my card, or if it is lost?',
  'When did you send me my new card?',
  'Do you have info about the card on delivery?',
  'What do I do if I still have not received my new card?',
  'Does the package with my card have tracking?',
  "I ordered my card but it still isn't here",
  'Why has my new card still not come?',
  "I still haven't received my card after two weeks, is it lost?",
  'Can you track my card for me?',
  'Is there a way to track the delivery of my card?',
  "It's been a week since I ordered my card and it's not here. Please help?",
  'Will I be able to track the card that was sent to me?'],
 'labels': tensor([12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12])}

## Detectors

### Instantiate and fit

In [12]:
detectors: List[ScorerType] = [MahalanobisScorer(layers=[-1])]

In [13]:
detectors = prepare_detectors(detectors, model, train_loader, tokenizer)

AttributeError: 'SequenceClassifierOutput' object has no attribute 'sequences'

### Evaluate

#### IN dataset

In [ ]:
print("Evaluating on the in-distribution training set")

records = evaluate_dataloader(
    model,
    train_loader,
    tokenizer,
    detectors
)

In [ ]:
print("Evaluating on the in-distribution test set")

records = evaluate_dataloader(
    model,
    test_loader,
    tokenizer,
    detectors
)